In [1]:
import omf
import pandas as pd
import numpy as np

In [5]:
# csv_file = "../design_assets/marvin.csv"
csv_file = ""
df = pd.read_csv(csv_file)
df = df.sort_values(by=["Z", "Y", "X"], ascending=[True, True, True])

df


,X,Y,Z,classification_no,size (X),size(Y),size(Z),classification,co,cu,density,domain,exclusion_zone,geotech_zone,ni,percentage,strength,weather
0,295270.0,6796670.0,-390.0,1,20.0,20.0,20.0,4,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
2048,295290.0,6796670.0,-390.0,1,20.0,20.0,20.0,4,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
256,295310.0,6796670.0,-390.0,1,20.0,20.0,20.0,4,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
2304,295330.0,6796670.0,-390.0,1,20.0,20.0,20.0,4,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
32,295350.0,6796670.0,-390.0,1,20.0,20.0,20.0,4,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734301,297470.0,6799490.0,490.0,1,20.0,20.0,20.0,4,0.000,0.000,0.0,ng,0,40,0.000,0.0,AIR,Air
734303,297490.0,6799490.0,490.0,1,20.0,20.0,20.0,4,0.000,0.000,0.0,ng,0,40,0.000,0.0,AIR,Air
734845,297510.0,6799490.0,490.0,1,20.0,20.0,20.0,4,0.000,0.000,0.0,ng,0,40,0.000,0.0,AIR,Air
734847,297530.0,6799490.0,490.0,1,20.0,20.0,20.0,4,0.000,0.000,0.0,ng,0,40,0.000,0.0,AIR,Air


In [24]:
# bm_origin = [3475, 6480, 285]
# min_x, max_x = 3475, 5305
# min_y, max_y = 6480, 8280
# min_z, max_z = 285, 795
# block_size = 30.
# block_size_z = 30.
bm_origin = [295260, 6796660, -400]
min_x, max_x = 295260, 297560
min_y, max_y = 6796660, 6799500
min_z, max_z = -400, 500
block_size = 20.
block_size_z = 10.
differenceX = max_x - min_x
differenceY = max_y - min_y
differenceZ = max_z - min_z
num_blocksX = differenceX // block_size
num_blocksY = differenceY // block_size
num_blocksZ = differenceZ // block_size_z

tensor_u = np.full(int(num_blocksX), block_size, dtype=float)
tensor_v = np.full(int(num_blocksY), block_size, dtype=float)
tensor_w = np.full(int(num_blocksZ), block_size_z, dtype=float)


In [20]:
print(num_blocksX, num_blocksY, num_blocksZ)

61.0 60.0 17.0


In [25]:
bm_attributes = []

# for element in ["block_value", "copper", "gold", "sg"]:
#   if element in df.columns:
#       element_array = df[element].astype(float).to_numpy()
#       attr = omf.ScalarData(
#          array = np.array(element_array), name =element,  location='cells', )
#       bm_attributes.append(attr)

# for element in ["air", "classification_no"]:
#   if element in df.columns:
#       element_array = df[element].astype(int).to_numpy()
#       attr = omf.ScalarData(
#          array = np.array(element_array), name =element,  location='cells', )
#       bm_attributes.append(attr)
for element in ["co", "cu", "density", "percentage"]:
  if element in df.columns:
      element_array = df[element].astype(float).to_numpy()
      attr = omf.ScalarData(
         array = np.array(element_array), name =element,  location='cells', )
      bm_attributes.append(attr)

In [26]:
proj = omf.Project(
    name='test_project',
    description='Test OMF BM from CSV'
)
vol = omf.VolumeElement(
    name='test_bm',
    geometry=omf.VolumeGridGeometry(
        tensor_u=tensor_u,
        tensor_v=tensor_v,
        tensor_w=tensor_w,
        origin=bm_origin
    ),
    data=bm_attributes
)
# proj = omf.Project(
#     name='Marvin',
#     description='Marvin OMF BM from CSV'
# )
# vol = omf.VolumeElement(
#     name='marvin_bm',
#     geometry=omf.VolumeGridGeometry(
#         tensor_u=tensor_u,
#         tensor_v=tensor_v,
#         tensor_w=tensor_w,
#         origin=bm_origin
#     ),
#     data=bm_attributes
# )

In [22]:
all_vertex_coords = [[0,0,0], [0,1,0], [1,1,0], [1,0,-0.1]]
triangles = [[0,1,2],[2,3,0]]

In [23]:
grid = omf.SurfaceElement(
    name='trisurf',
    geometry=omf.SurfaceGeometry(
        vertices=np.array(all_vertex_coords),
        triangles=np.array(triangles)
    ),
)

In [27]:
proj.elements = [vol]
assert proj.validate()
omf.OMFWriter(proj, '../design_assets/test_bm_from_csv.omf')